In [41]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw/MD code."></form>''')



In [2]:
import cplex
from random import randint
from itertools import permutations, repeat
from pprint import pprint

In [3]:
# Inputs
routes = ["R1", "R2"]
stops = [["A", "B", "C", "D"],["AA","B","BB","C","CC"]]
buses = [6, 6]
transfers = []
omega = {}

for r1,_ in enumerate(routes):
    for r2,_ in enumerate(routes):
        if r2 > r1:
            common_stops = list(set(stops[r1]).intersection(stops[r2]))
            
            for stop in common_stops:
                r1_last_stop = (stops[r1].index(stop) == len(stops[r1])-1)
                r2_last_stop = (stops[r2].index(stop) == len(stops[r2])-1)
                
                if not (r1_last_stop and r2_last_stop):
                    if not r1_last_stop:
                        transfers.append((r2, r1, stop))
                        if r2 not in omega:                            
                            omega[r2] = {}
                        
                        if stop not in omega[r2]:
                            omega[r2][stop] = {}
                            
                        omega[r2][stop][r1] = randint(2,4)
                    
                    if not r2_last_stop:
                        transfers.append((r1, r2, stop))
                        
                        if r1 not in omega:                            
                            omega[r1] = {}
                        
                        if stop not in omega[r1]:
                            omega[r1][stop] = {}
                            
                        omega[r1][stop][r2] = randint(2,4)

sigma_f = [2,3]
sigma_l = [50,50]

delta_min = [5,5]
delta_max = [25,15]

dwell_max = [2,2]

plan_horizon = 100

# M = max(delta_max) + max([t for route in omega.values() for stop in route.values() for t in stop.values()])
M = plan_horizon

tt = []
for r,_ in enumerate(routes):
    tt.append([])
    for s, _ in enumerate(stops[r]):
        if s != len(stops[r]) - 1:
            tt[r].append([])
            for n in range(plan_horizon):
                tt[r][s].append(randint(2, 3))

groups = [([0,1], ['C'], 'A'), ([1,0], ['B'], 'AA'), ([0,1, 0], ['B', 'C'], 'A'), ([1,0], ['C'], 'AA')]

# demands = [randint(1,100) for _ in groups]
demands = [10, 56, 60, 52]
# alpha = [randint(1, plan_horizon) for _ in groups]
alpha = [10, 37, 40, 13]

In [4]:
cp = cplex.Cplex()

cp.objective.set_sense(cp.objective.sense.minimize)
cp.set_problem_type(cp.problem_type.LP)
cp.parameters.conflict.display = 2

idxFirst = {}
constraints = []

### Definition of variables

$d_{rs}^k$: `d[r][s][k]`

$a_{rs}^k$: `a[r][s][k]`

$h_{rs}^k$: `h[r][s][k]`

$d_{rs}^{kn}$: `db[r][s][k][n]`

$c_{rks}^{r^\prime k^\prime}$: `c[r][r'][s][k][k']`

$t_{rks}^{r^\prime k^\prime}$: `t[r][r'][s][k][k']`

$g_{grks}^{r^\prime k^\prime}$: `t[r][r'][s][k][k'][g]`

$\tilde{c}_g^k$: `co[g][k]`

$t_g$: `t[g]`

In [5]:
# Generate d, a h
varnames = []
d = []
a = []
h = []

idx = 0
for r, _ in enumerate(routes):
    d.append([])
    a.append([])
    h.append([])
    
    for s, _ in enumerate(stops[r]):
        d[r].append([])
        a[r].append([])
        h[r].append([])
        
        for k in range(buses[r]):
            d[r][s].append(idx)
            a[r][s].append(idx+1)
            h[r][s].append(idx+2)
            
            varnames.append("d_r{},s{},k{}".format(r,s,k))
            varnames.append("a_r{},s{},k{}".format(r,s,k))
            varnames.append("h_r{},s{},k{}".format(r,s,k))
            
            idx += 3

cp.variables.add(obj=[0] * len(varnames),
                types=[cp.variables.type.integer] * len(varnames),
                ub=[plan_horizon] * len(varnames))
cp.variables.set_names([(i,vn) for i,vn in enumerate(varnames)])

# Generate d, a h
varnames = []
db = []
idx_db1 = idx

for r, _ in enumerate(routes):
    db.append([])
    
    for s, _ in enumerate(stops[r]):
        db[r].append([])
        
        for k in range(buses[r]):
            db[r][s].append([])
            
            for n in range(plan_horizon):
                db[r][s][k].append(idx)
    
                varnames.append("db_r{},s{},k{},n{}".format(r,s,k,n))
        
                idx += 1

cp.variables.add(obj=[0] * len(varnames),
                types=[cp.variables.type.binary] * len(varnames))
cp.variables.set_names([(idx_db1+i,vn) for i,vn in enumerate(varnames)])

# Generate c, t, p
varnames = []
vartypes = []
obj = []
c = {}
t = {}
p = {}
zt = {}
idx_c1 = idx

for (r1, r2, s) in transfers:
    if r1 not in c: 
        c[r1] = {}
        t[r1] = {}
        p[r1] = {}
        zt[r1] = {}
    if r2 not in c[r1]: 
        c[r1][r2] = {}
        t[r1][r2] = {}
        p[r1][r2] = {}
        zt[r1][r2] = {}
    if s not in c[r1][r2]:
        c[r1][r2][s] = {}
        t[r1][r2][s] = {}
        p[r1][r2][s] = {}
        zt[r1][r2][s] = {}
    
    for k1, k2 in [(x,y) for x in range(buses[r1]) for y in range(buses[r2])]:
        if k1 not in c[r1][r2][s]:
            c[r1][r2][s][k1] = {}
            t[r1][r2][s][k1] = {}
            p[r1][r2][s][k1] = {}
            zt[r1][r2][s][k1] = {}
            
        c[r1][r2][s][k1][k2] = idx
        t[r1][r2][s][k1][k2] = idx+1
        varnames.append("c_r{},k{},s{},rp{},kp{}".format(r1,k1,s,r2,k2))
        varnames.append("t_r{},k{},s{},rp{},kp{}".format(r1,k1,s,r2,k2))
        
        obj.append(0)
        obj.append(0.000001)
        vartypes.append(cp.variables.type.binary)
        vartypes.append(cp.variables.type.integer)
        
        idx += 2
        
        p[r1][r2][s][k1][k2] = []
        zt[r1][r2][s][k1][k2] = []
        for g,_ in enumerate(groups):
            p[r1][r2][s][k1][k2].append(idx)
            zt[r1][r2][s][k1][k2].append(idx+1)
            varnames.append("p_g{},r{},k{},s{},rp{},kp{}".format(g,r1,k1,s,r2,k2))
            varnames.append("zt_g{},r{},k{},s{},rp{},kp{}".format(g,r1,k1,s,r2,k2))
            
            obj.append(0)
            obj.append(0)
            vartypes.append(cp.variables.type.binary)
            vartypes.append(cp.variables.type.integer)
            
            idx += 2
            
cp.variables.add(obj=obj,
                types=vartypes)
cp.variables.set_names([(i+idx_c1,vn) for i,vn in enumerate(varnames)])

# Generate co, z
varnames = []
vartypes = []
co = []
z = []
tg = []
idx_co1 = idx

for g, (rs, ss, o) in enumerate(groups):
    co.append([])
    z.append([])
    
    for k in range(buses[rs[0]]):
        co[g].append(idx)
        
        varnames.append("co_g{},k{}".format(g,k))
        vartypes.append(cp.variables.type.binary)
        
        idx += 1
        
        z[g].append([])
        for kp in range(buses[rs[1]]):
            z[g][k].append(idx)
            
            varnames.append("z_g{},k{},kp{}".format(g,k,kp))
            vartypes.append(cp.variables.type.binary)
            
            idx += 1
        
cp.variables.add(obj=[0] * len(varnames),
                types=vartypes)
cp.variables.set_names([(i+idx_co1,vn) for i,vn in enumerate(varnames)])

# Generate tg
varnames = []
tg = []
obj = []
idx_tg1 = idx

for g, (rs, ss, o) in enumerate(groups):
    tg.append(idx)
    obj.append(demands[g])
    
    varnames.append("tg_g{}".format(g))
    idx += 1
    
cp.variables.add(obj=obj,
                types=[cp.variables.type.integer] * len(varnames))
cp.variables.set_names([(i+idx_tg1,vn) for i,vn in enumerate(varnames)])

\begin{align}
    &d_{r1}^1 = \sigma_r^f  &\forall r \in \mathcal{R}
    \tag{2}
\end{align}

In [6]:
lc = cp.linear_constraints.get_num()
constraints = []

for r, _ in enumerate(routes):
    constraints.append([[d[r][0][0]],[1]])
        
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['E'] * len(constraints),
                          rhs=sigma_f)
cp.linear_constraints.set_names([(lc+i,"C2({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    &d_{r1}^K = \sigma_r^l  &\forall r \in \mathcal{R}
    \tag{3}
\end{align}

In [7]:
lc = cp.linear_constraints.get_num()
constraints = []

for r, _ in enumerate(routes):
    constraints.append([[d[r][0][len(d[r][0])-1]],[1]])
        
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['E'] * len(constraints),
                          rhs=sigma_l)
cp.linear_constraints.set_names([(lc+i,"C3({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    &\underline{\delta}_r \leq d_{rs}^{k+1} - d_{rs}^k \leq \bar{\delta}_r  &\forall r \in \mathcal{R}, s \in \mathcal{S}_r, k = 1,2, \dots , k_r - 1
    \tag{4}
\end{align}

In [8]:
lc = cp.linear_constraints.get_num()
constraints = []

rs = []
rv = []
for r, _ in enumerate(routes):
    for s, _ in enumerate(stops[r]):
        for k in range(buses[r]-1):
            constraints.append([[d[r][s][k+1], d[r][s][k]],[1,-1]])

            rs.append(delta_max[r])
            rv.append(delta_min[r] - delta_max[r])

cp.linear_constraints.add(lin_expr=constraints,
                          senses=['R'] * len(constraints),
                          rhs=rs,
                          range_values=rv)
cp.linear_constraints.set_names([(lc+i,"C4({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    &d_{rs}^{k+1} \geq d_{rs}^k &\forall r \in \mathcal{R}, k = 1,2,\dots ,k_r - 1
    \tag{5}
\end{align}

In [9]:
lc = cp.linear_constraints.get_num()
constraints = []

for r, _ in enumerate(routes):
#     for s, _ in enumerate(stops[r]):
    for k in range(buses[r]-1):
        constraints.append([[d[r][0][k], d[r][0][k+1]],[1,-1]])
        
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['L'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C2.1({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    &d_{rs}^k = a_{rs}^k + h_{rs}^k  &\forall r \in \mathcal{R}, s \in \mathcal{S}_r, k =1,2,\dots ,k_r
    \tag{6}
\end{align}

In [10]:
lc = cp.linear_constraints.get_num()
constraints = []

for r, _ in enumerate(routes):
    for s, _ in enumerate(stops[r]):
        for k in range(buses[r]):
            constraints.append([[d[r][s][k], a[r][s][k], h[r][s][k]], [1,-1,-1]])

cp.linear_constraints.add(lin_expr=constraints,
                          senses=['E'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C6({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    &0 \leq h_{rs}^k \leq \eta_r  &\forall r \in \mathcal{R}, s \in \mathcal{S}_r, k =1,2,\dots ,k_r
    \tag{7}
\end{align}

In [11]:
lc = cp.linear_constraints.get_num()
constraints = []
rs = []

for r, _ in enumerate(routes):
    for s, _ in enumerate(stops[r]):
        for k in range(buses[r]):
            constraints.append([[h[r][s][k]], [1]])
            rs.append(dwell_max[r])

cp.linear_constraints.add(lin_expr=constraints,
                          senses=['R'] * len(constraints),
                          rhs=rs,
                          range_values=[-1 * x for x in rs])
cp.linear_constraints.set_names([(lc+i,"C7({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    &a_{r(s+1)}^k = d_{rs}^k + \theta_{rs}^{d_{rs}^k}  &\forall r \in \mathcal{R}, s \in \mathcal{S}_r, k \in \mathcal{K}_r
    \tag{8}\\
    &a_{r(s+1)}^k = d_{rs}^k + \sum_{n < H} \tilde{d}_{rs}^{kn} \theta_{rs}^n
    \tag{11}
\end{align}

In [12]:
lc = cp.linear_constraints.get_num()
constraints = []

for r, _ in enumerate(routes):
    for s in range(len(stops[r])-1):
        for k in range(buses[r]):
            ind = [a[r][s+1][k], d[r][s][k]]
            val = [1,-1]
            
            for n in range(plan_horizon):
                ind.append(db[r][s][k][n])
                val.append(-1*tt[r][s][n])
            
            constraints.append([ind, val])
        
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['E'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C11({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    &d_{rs}^k = \sum n \cdot \tilde{d}_{rs}^{kn}  &&\forall r,s,k
    \tag{9-}
\end{align}

In [13]:
lc = cp.linear_constraints.get_num()
constraints = []

for r, _ in enumerate(routes):
    for s in range(len(stops[r])):
        for k in range(buses[r]):
            ind = [d[r][s][k]]
            val = [1]
            
            for n in range(plan_horizon):
                ind.append(db[r][s][k][n])
                val.append(-1*n)
            
            constraints.append([ind, val])
        
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['E'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C9({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    &\sum_n \tilde{d}_{rs}^{kn} = 1  &&\forall r,s,k
    \tag{10}
\end{align}

In [14]:
lc = cp.linear_constraints.get_num()
constraints = []

for r, _ in enumerate(routes):
    for s in range(len(stops[r])):
        for k in range(buses[r]):
            ind = []
            
            for n in range(plan_horizon):
                ind.append(db[r][s][k][n])
            
            constraints.append([ind, [1]*len(ind)])
        
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['E'] * len(constraints),
                          rhs=[1] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C10({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    &M(c_{rks}^{r^\prime k^\prime} - 1) \leq d_{r^\prime s}^{k^\prime} - (a_{rs}^k + \omega_{rs}^{r^\prime}) \leq Mc_{rks}^{r^\prime k^\prime}
    \tag{12}
\end{align}

In [15]:
lc = cp.linear_constraints.get_num()
constraints = []
rhs = []

for (r, rp, s) in transfers:
    stop_r = stops[r].index(s)
    stop_rp = stops[rp].index(s)
    
    for k, kp in [(x,y) for x in range(buses[r]) for y in range(buses[rp])]:
        ind = [d[rp][stop_rp][kp], a[r][stop_r][k], c[r][rp][s][k][kp]]
        val = [1,-1,-M]
        
        constraints.append([ind, val])
        rhs.append(omega[r][s][rp])

cp.linear_constraints.add(lin_expr=constraints,
                          senses=['R'] * len(constraints),
                          rhs=rhs,
                          range_values=[-1*M] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C12({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    d_{r^\prime s}^{k^\prime} - (a_{rs}^k + \omega_{rs}^{r^\prime}) - Mc_{rks}^{r^\prime (k^\prime-1)} \leq t_{rks}^{r^\prime k^\prime}
    \tag{13}
\end{align}

In [16]:
lc = cp.linear_constraints.get_num()
constraints = []
rhs = []

for (r, rp, s) in transfers:
    stop_r = stops[r].index(s)
    stop_rp = stops[rp].index(s)
    
    for k, kp in [(x,y) for x in range(buses[r]) for y in range(buses[rp])]:
        if kp != 0:
            ind = [d[rp][stop_rp][kp], a[r][stop_r][k], c[r][rp][s][k][kp-1], t[r][rp][s][k][kp]]
            val = [1,-1,-M,-1]
        else:
            ind = [d[rp][stop_rp][kp], a[r][stop_r][k], t[r][rp][s][k][kp]]
            val = [1,-1,-1]
        
        constraints.append([ind, val])
        rhs.append(omega[r][s][rp])

cp.linear_constraints.add(lin_expr=constraints,
                          senses=['L'] * len(constraints),
                          rhs=rhs)
cp.linear_constraints.set_names([(lc+i,"C13({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    M(\tilde{c}_g^k - 1) \leq d_{r_1 s_o}^k - \alpha_g \leq M\tilde{c}_g^k
    \tag{14}
\end{align}

In [17]:
lc = cp.linear_constraints.get_num()
constraints = []
rhs = []
rv = []

for g, (rs, ss, o) in enumerate(groups):
    for k in range(buses[rs[0]]):
        ind = [d[rs[0]][stops[rs[0]].index(o)][k], co[g][k]]
        val = [1, -M]
        
        constraints.append([ind, val])
        rhs.append(alpha[g])
        rv.append(-1*plan_horizon)
    
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['R'] * len(constraints),
                          rhs=rhs,
                          range_values=rv)
cp.linear_constraints.set_names([(lc+i,"C14({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
	p_{gr_1ks_o}^{r_2k^\prime} = 
		\begin{cases}
			1,	&c_{r_1ks}^{r_2k^\prime} - c_{r_1ks}^{r_2 (k^\prime - 1)} = 1 \text{ and } \tilde{c}_g^k - \tilde{c}_g^{k-1} = 1\\
			0,	&\text{otherwise}
		\end{cases}
		\tag{15}
\end{align}

\begin{align}
    z &\geq \left(c_{r_1ks}^{r_2k^\prime} - c_{r_1ks}^{r_2 (k^\prime - 1)}\right) + \left(\tilde{c}_g^k - \tilde{c}_g^{k-1}\right) - 1 \tag{15a}\\
    z &\leq c_{r_1ks}^{r_2k^\prime} - c_{r_1ks}^{r_2 (k^\prime - 1)} \tag{15b}\\
    z &\leq \tilde{c}_g^k - \tilde{c}_g^{k-1} \tag{15c}
\end{align}

\begin{align}
    p_{gr_1ks_o}^{r_2 k^\prime} \leq Mz \tag{15h}\\
    p_{gr_1ks_o}^{r_2 k^\prime} \geq z \tag{15i}
\end{align}

In [18]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, _) in enumerate(groups):
    r1 = rs[0]
    r2 = rs[1]
    
    for k, kp in [(x,y) for x in range(buses[r1]) for y in range(buses[r2])]:
        if k == 0 and kp == 0:
            ind = [z[g][k][kp], c[r1][r2][ss[0]][k][kp], co[g][k]]
            val = [1,-1,-1]
        elif kp == 0:
            ind = [z[g][k][kp], c[r1][r2][ss[0]][k][kp], co[g][k], co[g][k-1]]
            val = [1,-1,-1, 1]
        elif k == 0:
            ind = [z[g][k][kp], c[r1][r2][ss[0]][k][kp], c[r1][r2][ss[0]][k][kp-1], co[g][k]]
            val = [1,-1,1,-1]
        else:
            ind = [z[g][k][kp], c[r1][r2][ss[0]][k][kp], c[r1][r2][ss[0]][k][kp-1], co[g][k], co[g][k-1]]
            val = [1,-1,1,-1,1]
            
        constraints.append([ind, val])
            
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['G'] * len(constraints),
                          rhs=[-1] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C15a({})".format(i)) for i,_ in enumerate(constraints)])

In [19]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, _) in enumerate(groups):
    r1 = rs[0]
    r2 = rs[1]
    
    for k, kp in [(x,y) for x in range(buses[r1]) for y in range(buses[r2])]:
        if kp == 0:
            ind = [z[g][k][kp], c[r1][r2][ss[0]][k][kp]]
            val = [1,-1]
        else:
            ind = [z[g][k][kp], c[r1][r2][ss[0]][k][kp], c[r1][r2][ss[0]][k][kp-1]]
            val = [1,-1,1]
            
        constraints.append([ind, val])
            
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['L'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C15b({})".format(i)) for i,_ in enumerate(constraints)])

In [20]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, _) in enumerate(groups):
    r1 = rs[0]
    r2 = rs[1]
    
    for k, kp in [(x,y) for x in range(buses[r1]) for y in range(buses[r2])]:
        if k == 0:
            ind = [z[g][k][kp], co[g][k]]
            val = [1,-1]
        else:
            ind = [z[g][k][kp], co[g][k], co[g][k-1]]
            val = [1,-1,1]
            
        constraints.append([ind, val])
            
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['L'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C15c({})".format(i)) for i,_ in enumerate(constraints)])

In [21]:
# lc = cp.linear_constraints.get_num()
# constraints = []

# for g, (rs, ss, o) in enumerate(groups):
#     r1 = rs[0]
#     r2 = rs[1]
    
#     for k, kp in [(x,y) for x in range(buses[r1]) for y in range(buses[r2])]:
#         ind = [p[r1][r2][ss[0]][k][kp][g], z[g][k][kp]]
#         val = [1,M]

#         constraints.append([ind, val])
            
# cp.linear_constraints.add(lin_expr=constraints,
#                           senses=['L'] * len(constraints),
#                           rhs=[1+M] * len(constraints))
# cp.linear_constraints.set_names([(lc+i,"C3.4d({})".format(i)) for i,_ in enumerate(constraints)])

In [22]:
# lc = cp.linear_constraints.get_num()
# constraints = []

# for g, (rs, ss, o) in enumerate(groups):
#     r1 = rs[0]
#     r2 = rs[1]
    
#     for k, kp in [(x,y) for x in range(buses[r1]) for y in range(buses[r2])]:
#         ind = [p[r1][r2][ss[0]][k][kp][g], z[g][k][kp]]
#         val = [1,-M]
            
#         constraints.append([ind, val])
            
# cp.linear_constraints.add(lin_expr=constraints,
#                           senses=['G'] * len(constraints),
#                           rhs=[1-M] * len(constraints))
# cp.linear_constraints.set_names([(lc+i,"C3.4e({})".format(i)) for i,_ in enumerate(constraints)])

In [23]:
# lc = cp.linear_constraints.get_num()
# constraints = []

# for g, (rs, ss, o) in enumerate(groups):
#     r1 = rs[0]
#     r2 = rs[1]
    
#     for k, kp in [(x,y) for x in range(buses[r1]) for y in range(buses[r2])]:
#         ind = [p[r1][r2][ss[0]][k][kp][g], z[g][k][kp]]
#         val = [1,-M]
            
#         constraints.append([ind, val])
            
# cp.linear_constraints.add(lin_expr=constraints,
#                           senses=['L'] * len(constraints),
#                           rhs=[0] * len(constraints))
# cp.linear_constraints.set_names([(lc+i,"C3.4f({})".format(i)) for i,_ in enumerate(constraints)])

In [24]:
# lc = cp.linear_constraints.get_num()
# constraints = []

# for g, (rs, ss, o) in enumerate(groups):
#     r1 = rs[0]
#     r2 = rs[1]
    
#     for k, kp in [(x,y) for x in range(buses[r1]) for y in range(buses[r2])]:
#         ind = [p[r1][r2][ss[0]][k][kp][g], z[g][k][kp]]
#         val = [1,M]
            
#         constraints.append([ind, val])
            
# cp.linear_constraints.add(lin_expr=constraints,
#                           senses=['G'] * len(constraints),
#                           rhs=[0] * len(constraints))
# cp.linear_constraints.set_names([(lc+i,"C3.4e({})".format(i)) for i,_ in enumerate(constraints)])

In [25]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, o) in enumerate(groups):
    r1 = rs[0]
    r2 = rs[1]
    
    for k, kp in [(x,y) for x in range(buses[r1]) for y in range(buses[r2])]:
        ind = [p[r1][r2][ss[0]][k][kp][g], z[g][k][kp]]
        val = [1,-1*M]
            
        constraints.append([ind, val])
            
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['L'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C15h({})".format(i)) for i,_ in enumerate(constraints)])

In [26]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, o) in enumerate(groups):
    r1 = rs[0]
    r2 = rs[1]
    
    for k, kp in [(x,y) for x in range(buses[r1]) for y in range(buses[r2])]:
        ind = [p[r1][r2][ss[0]][k][kp][g], z[g][k][kp]]
        val = [1,-1]
            
        constraints.append([ind, val])
            
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['G'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C15i({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    &\sum_k \sum_{k^\prime} p_{grks}^{r^\prime k^\prime} = 1 	&&\forall (r, r^\prime, s) \in \mathcal{T}_g
    \tag{C3.5}
\end{align}

In [27]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, o) in enumerate(groups):
    trs = zip(rs[:-1], rs[1:], ss)
    
    for r, rp, s in trs:
        ind = []
        
        for k, kp in [(x,y) for x in range(buses[r]) for y in range(buses[rp])]:
            ind.append(p[r][rp][s][k][kp][g])
            
        constraints.append([ind, [1] * len(ind)])
        
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['E'] * len(constraints),
                          rhs=[1] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C3.5({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
    &\sum_{k^\prime} k^\prime \cdot p_{grks}^{r^\prime k^\prime} = \sum_{k^\prime} k^\prime \cdot p_{gr^\prime k^\prime s^\prime}^{r^{\prime\prime} k^{\prime\prime}}
    \tag{C3.6}
\end{align}

In [28]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, o) in enumerate(groups):
    trs = zip(rs[:-1], rs[1:], ss)
    
    for (r1, rp1, s1), (r2, rp2, s2) in zip(trs[:-1], trs[1:]):
        ind = []
        val = []

        for k1, kp1 in [(x,y) for x in range(buses[r1]) for y in range(buses[rp1])]:
            ind.append(p[r1][rp1][s1][k1][kp1][g])
            val.append(kp1+1)

        for k2, kp2 in [(x,y) for x in range(buses[r2]) for y in range(buses[rp2])]:
            ind.append(p[r2][rp2][s2][k2][kp2][g])
            val.append(-k2-1)

        constraints.append([ind, val])

if len(constraints) > 0:        
    cp.linear_constraints.add(lin_expr=constraints,
                              senses=['E'] * len(constraints),
                              rhs=[0] * len(constraints))
    cp.linear_constraints.set_names([(lc+i,"C3.6({})".format(i)) for i,_ in enumerate(constraints)])

In [29]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, o) in enumerate(groups):
    trs = zip(rs[:-1], rs[1:], ss)
    
    for r, rp, s in trs:
        for k, kp in [(x,y) for x in range(buses[r]) for y in range(buses[rp])]:
            ind = [p[r][rp][s][k][kp][g], c[r][rp][s][k][kp]]
            val = [1,-1]
            
            constraints.append([ind, val])
        
cp.linear_constraints.add(lin_expr=constraints,
                          senses=['L'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C3.8({})".format(i)) for i,_ in enumerate(constraints)])

\begin{align}
	t_g = \sum_{(r, r^\prime, s)} \sum_{k \in \mathcal{K}_r} \sum_{k^\prime \in \mathcal{K}_{r^\prime}} p_{grks}^{r^\prime k^\prime} t_{rks}^{r^\prime k^\prime}
    \tag{C3.7}
\end{align}

\begin{align}
    z_{grks}^{r^\prime k^\prime} &\leq t_h p_{grks}^{r^\prime k^\prime} \tag{C3.7a}\\
    z_{grks}^{r^\prime k^\prime} &\geq 0 \tag{C3.7b} \\
    z_{grks}^{r^\prime k^\prime} &\geq t_{rks}^{r^\prime k^\prime} - t_h(1-p_{grks}^{r^\prime k^\prime}) \tag{C3.7c}\\
    z_{grks}^{r^\prime k^\prime} &\leq t_{rks}^{r^\prime k^\prime} \tag{C3.7d}\\
    t_g &= \sum \sum \sum z_{grks}^{r^\prime k^\prime} \tag{C3.7e}
\end{align}

In [30]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, _) in enumerate(groups):
    trs = zip(rs[:-1], rs[1:], ss)

    for r, rp, s in trs:
        for k, kp in [(x,y) for x in range(buses[r]) for y in range(buses[rp])]:
            ind = [zt[r][rp][s][k][kp][g], p[r][rp][s][k][kp][g]]
            val = [1, -1*plan_horizon]
            
            constraints.append([ind, val])

cp.linear_constraints.add(lin_expr=constraints,
                          senses=['L'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C3.7a({})".format(i)) for i,_ in enumerate(constraints)])

In [31]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, _) in enumerate(groups):
    trs = zip(rs[:-1], rs[1:], ss)

    for r, rp, s in trs:
        for k, kp in [(x,y) for x in range(buses[r]) for y in range(buses[rp])]:
            ind = [zt[r][rp][s][k][kp][g], t[r][rp][s][k][kp], p[r][rp][s][k][kp][g]]
            val = [1, -1, -1*plan_horizon]
            
            constraints.append([ind, val])

cp.linear_constraints.add(lin_expr=constraints,
                          senses=['G'] * len(constraints),
                          rhs=[-1*plan_horizon] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C3.7c({})".format(i)) for i,_ in enumerate(constraints)])

In [32]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, _) in enumerate(groups):
    trs = zip(rs[:-1], rs[1:], ss)

    for r, rp, s in trs:
        for k, kp in [(x,y) for x in range(buses[r]) for y in range(buses[rp])]:
            ind = [zt[r][rp][s][k][kp][g], t[r][rp][s][k][kp]]
            val = [1, -1]
            
            constraints.append([ind, val])

cp.linear_constraints.add(lin_expr=constraints,
                          senses=['L'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C3.7d({})".format(i)) for i,_ in enumerate(constraints)])

In [33]:
lc = cp.linear_constraints.get_num()
constraints = []

for g, (rs, ss, _) in enumerate(groups):
    trs = zip(rs[:-1], rs[1:], ss)

    ind = [tg[g]]
    val = [1]
    
    for r, rp, s in trs:
        for k, kp in [(x,y) for x in range(buses[r]) for y in range(buses[rp])]:
            ind.append(zt[r][rp][s][k][kp][g])
            val.append(-1)
            
    constraints.append([ind, val])

cp.linear_constraints.add(lin_expr=constraints,
                          senses=['E'] * len(constraints),
                          rhs=[0] * len(constraints))
cp.linear_constraints.set_names([(lc+i,"C3.7e({})".format(i)) for i,_ in enumerate(constraints)])

In [34]:
cp.write("model_v0_oct12.lp")

In [35]:
try:
    cp.solve()
except cplex.exceptions.CplexError as exc:
    print(exc)

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 3 times.
MIP Presolve eliminated 1271 rows and 5753 columns.
MIP Presolve modified 2793 coefficients.
Aggregator did 78 substitutions.
Reduced MIP has 730 rows, 1610 columns, and 4766 nonzeros.
Reduced MIP has 1223 binaries, 271 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (29.03 ticks)
Probing fixed 229 vars, tightened 113 bounds.
Probing changed sense of 11 constraints.
Probing time = 0.08 sec. (27.18 ticks)
Cover probing fixed 2 vars, tightened 67 bounds.
Tried aggregator 2 times.
MIP Presolve eliminated 152 rows and 335 columns.
MIP Presolve modified 503 coefficients.
Aggregator did 4 substitutions.
Reduced MIP has 574 rows, 1271 columns, and 3715 nonzeros.
Reduced MIP has 946 binaries, 227 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (8.26 ticks)
Probing fixed 13 vars, tight

In [36]:
# pprint(zip(cp.solution.get_values(), cp.variables.get_names()))
print cp.solution.get_objective_value()
v = cp.solution.get_values()
            
print alpha, demands
print transfers
# for r, rp, s in transfers:
#     print r, rp, s
#     for k in range(buses[r]):
#         print "\t".join([str(int(v[t[r][rp][s][k][kp]])) if int(v[c[r][rp][s][k][kp]]) else "." for kp in range(buses[rp])])
    
print " "

for g, (rs, ss, o) in enumerate(groups):
    print "Group {}: t_g={} alpha_g={}".format(g, v[tg[g]], alpha[g])
    
    trs = zip(rs[:-1], rs[1:], ss)
    
    s_prev = False
    for i, (r, rp, s) in enumerate(trs):
#         for k in range(buses[r]):
#             print "\t".join([str(int(v[p[r][rp][s][k][kp][g]])) for kp in range(buses[rp])])
        
        for k, kp in [(x,y) for x in range(buses[r]) for y in range(buses[rp])]:
            if abs(v[p[r][rp][s][k][kp][g]] - 1) <= 0.001:
                break

        if i == 0:
            print v[d[r][stops[r].index(o)][k]], v[a[r][stops[r].index(s)][k]]
        else:
            print v[d[r][stops[r].index(s_prev)][k]], v[a[r][stops[r].index(s)][k]]
        
        s_prev = s
    
    print v[d[rp][stops[rp].index(s)][kp]], plan_horizon
    print '------------------------------------------------------------'
            
print ""
        
for r,_ in enumerate(routes):
    print "Route {}:\n".format(r)
    print "\t".join(stops[r])
    print "-" * (len(stops[r])*8)
    for k in range(buses[r]):
        print "\t".join([str(int(v[d[r][s][k]])) for s,_ in enumerate(stops[r])])
        
    print ""
    
for r,_ in enumerate(routes):
    print "Route {}:\n".format(r)
    print "\t".join(stops[r])
    print "-" * (len(stops[r])*8)
    for k in range(buses[r]):
        print "\t".join([str(int(v[a[r][s][k]])) for s,_ in enumerate(stops[r])])
        
    print ""
        

2e-05
[10, 37, 40, 13] [10, 56, 60, 52]
[(1, 0, 'C'), (0, 1, 'C'), (1, 0, 'B'), (0, 1, 'B')]
 
Group 0: t_g=0.0 alpha_g=10
10.0 17.0
20.0 100
------------------------------------------------------------
Group 1: t_g=0.0 alpha_g=37
42.0 45.0
47.0 100
------------------------------------------------------------
Group 2: t_g=0.0 alpha_g=40
43.0 45.0
47.0 53.0
55.0 100
------------------------------------------------------------
Group 3: t_g=0.0 alpha_g=13
22.0 33.0
35.0 100
------------------------------------------------------------

Route 0:

A	B	C	D
--------------------------------
2	7	11	14
10	15	17	19
21	26	28	30
28	32	35	38
43	47	49	53
50	53	55	60

Route 1:

AA	B	BB	C	CC
----------------------------------------
3	7	11	13	16
9	15	18	20	22
22	26	30	33	37
27	32	36	39	43
42	47	51	53	55
50	53	56	58	60

Route 0:

A	B	C	D
--------------------------------
2	5	10	13
8	13	17	19
21	24	28	30
28	30	35	38
41	45	49	52
50	52	55	58

Route 1:

AA	B	BB	C	CC
----------------------------------------
3	5

In [37]:
from plotly import __version__
from plotly.tools import set_credentials_file
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout

set_credentials_file(username='kelvinlee18', api_key='VEAEhweIN3AloAbkxPD4')
init_notebook_mode(connected=True)

layout_default = Layout(
    height=200,
    showlegend = False,
    xaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    ), yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgb(.95,.95,.95)',
        showline=False,
        ticks='',
        showticklabels=False
    )
)

def alter_layout(modifications):
    layout = layout_default
    for k, v in modifications.iteritems():
        if isinstance(v, dict):
            for k2, v2 in modifications[k].iteritems():
                layout[k][k2] = v2
        else:
            layout[k] = v
        
    return layout

In [38]:
departures = []
arrivals = []

for r, _ in enumerate(routes):
    departures.append([])
    arrivals.append([])
    
    departures[r] = [v[d[r][0][k]] for k in range(buses[r])]
    arrivals[r] = [v[a[r][0][k]] for k in range(buses[r])]

x_departure = []
y_departure = []
text_departure = []

x_arrival = []
y_arrival = []
text_arrival = []

traces = []

for r, (departure, arrival) in enumerate(zip(departures, arrivals)):
    x_arrival += arrival
    y_arrival += [r] * len(arrival)
    text_arrival += ["<b>Route</b>: {}<br><b>Bus</b>: {}<br><b>Arrival</b>: {}<br><b>Departure</b>: {}".format(routes[r], k, int(arr), int(dep)) for dep, arr in zip(departure, arrival)]
    
    x_departure += departure
    y_departure += [r] * len(departure)
    text_departure += ["<b>Route</b>: {}<br><b>Bus</b>: {}<br><b>Arrival</b>: {}<br><b>Departure</b>: {}".format(routes[r], k, int(arr), int(dep)) for dep, arr in zip(departure, arrival)]
    
traces.append(Scatter(
    x = map(round, x_arrival),
    y = y_arrival,
    mode = 'markers',
    marker = dict(
        size = 20,
        color = [0] * len(x_arrival),
        colorscale = 'RdBu',
    ),
    hoverinfo = "text",
    text = text_departure,
    opacity = .3
))

traces.append(Scatter(
    x = map(round, x_departure),
    y = y_departure,
    mode = 'markers',
    marker = dict(
        symbol = "triangle-right",
        size = 10,
        color = y_departure,
        colorscale = 'Jet',
        cmin = 0,
        cmax = len(routes)-1
    ),
    hoverinfo = "text",
    text = text_arrival
))
                    
lo = alter_layout({
    'title'  : "Schedule",
    'hovermode': "closest",
    'height' : 100+80*len(routes),
    'yaxis'  : {'zeroline': False, 'showgrid': False, 'tickvals': range(len(routes)), 'ticktext': routes, 'showticklabels': True, 'autorange': 'reversed'},
    'xaxis'  : {'autorange': False, 'range': [-1, plan_horizon+1], 'showgrid': True, 'autotick': False, 'dtick': 5, 'showticklabels': True},
})

fig = Figure(data=traces, layout=lo)
iplot(fig, filename='ISTTT23_departures')

In [39]:
x_departure = []
y_departure = []
r_arrival = []
text_departure = []

x_arrival = []
y_arrival = []
r_departure = []
text_arrival = []

traces = []

for t, (r, rp, s) in enumerate(transfers):
    stop_r = stops[r].index(s)
    stop_rp = stops[rp].index(s)
    
    x_arrival += [v[a[r][stop_r][k]] for k in range(buses[r])]
    y_arrival += [t] * buses[r]
    r_arrival += [r] * buses[r]
    
    for k in range(buses[r]):
        match = 0
        for kt in range(buses[rp]):
            if abs(v[c[r][rp][s][k][kt]] - 1) <= 0.00001:
                text_arrival.append('<b>Arrival</b>: {}<br><b>Connects</b>:{}'.format(round(v[a[r][stop_r][k]]), routes[rp]+'-'+str(kt)))
                match = 1
                break
        
        if not match:
            text_arrival.append('<b>Arrival</b>: {}<br><b>Connects</b>:{}'.format(round(v[a[r][stop_r][k]]), 'NC'))
    
    x_departure += [v[d[rp][stop_rp][kp]] for kp in range(buses[rp])]
    y_departure += [t] * buses[rp]
    r_departure += [rp] * buses[rp]
    text_departure += ['{}-{}'.format(routes[rp], kp) for kp in range(buses[rp])]
    
traces.append(Scatter(
    x = map(round, x_arrival),
    y = y_arrival,
    mode = 'markers',
    marker = dict(
        size = 20,
        color = r_arrival,
        colorscale = 'Jet',
        cmin = 0,
        cmax = len(routes)-1
    ),
    hoverinfo = "text",
    text = text_arrival
))

traces.append(Scatter(
    x = map(round, x_departure),
    y = y_departure,
    mode = 'markers',
    marker = dict(
        symbol = "triangle-right",
        size = 10,
        color = r_departure,
        colorscale = 'Jet',
        cmin = 0,
        cmax = len(routes)-1
    ),
    hoverinfo = "text",
    text = text_departure
))
                    
lo = alter_layout({
    'title'  : "Transfers",
    'hovermode': "closest",
    'height' : 100+60*len(transfers),
    'yaxis'  : {'zeroline': False, 'showgrid': False, 'tickvals': range(len(transfers)), 'ticktext': ['{}_{}_{}'.format(routes[r], routes[rp], s) for r, rp, s in transfers], 'showticklabels': True, 'autorange': False},
    'xaxis'  : {'autorange': False, 'range': [-1, plan_horizon+1], 'showgrid': True, 'autotick': False, 'dtick': 5, 'showticklabels': True},
})

fig = Figure(data=traces, layout=lo)
iplot(fig, filename='ISTTT23_transfers')

In [40]:
from matplotlib import cm, colors
jet = cm.get_cmap('jet')
norm = colors.Normalize(vmin=0, vmax=len(routes)-1)

traces = []

traces.append(Scatter(
    x = alpha,
    y = range(len(groups)),
    mode = 'markers',
    marker = dict(
        size = 10,
        color = [0] * len(groups),
        colorscale = 'RdBu',
        cmin = -1,
        cmax = 1
    ),
    hoverinfo = "text",
#     text = ['<b>Arrival</b>: {}'.format(alpha[g]) for g in range(len(groups))]
))

for g, (rs, ss, o) in enumerate(groups):
    sp = o
    
    for r, rp, s in zip(rs[:-1], rs[1:], ss):
        stop_r = stops[r].index(s)
        stop_rs = stops[rp].index(s)
        
        for k, kp in [(k, kp)for k in range(buses[r]) for kp in range(buses[rp])]:
            if abs(v[p[r][rp][s][k][kp][g]] - 1) <= 0.0001:
                traces.append(Scatter(
                    x = [round(v[d[r][stops[r].index(sp)][k]]), 
                         round(v[a[r][stop_r][k]])],
                    y = [g, g],
                    mode = 'lines+markers',
                    marker = dict(
                        size = 10,
                        color = [r, r],
                        colorscale = 'Jet',
                        cmin = 0,
                        cmax = len(routes)-1,
                    ), line = dict(
                        width = 5,
                        color = 'rgb' + str(colors.colorConverter.to_rgb(jet(norm(r)))),
                    ), hoverinfo = "text",
                    text = ['<b>Bus</b>: R{}-{}<br><b>Departs</b>: {}'.format(r,k,round(v[d[r][stops[r].index(sp)][k]])),
                            '<b>Bus</b>: R{}-{}<br><b>Arrives</b>: {}<br><b>Transfer time</b>: {}'.format(r,k,round(v[a[r][stop_r][k]]),omega[r][s][rp])]
                ))
                
                break
                
        
                
        sp = s
        
    traces.append(Scatter(
        x = [round(v[d[rp][stops[rp].index(sp)][kp]]), 
             min([round(v[d[rp][stops[rp].index(sp)][kp]])+10, plan_horizon])],
        y = [g, g],
        mode = 'lines+markers',
        marker = dict(
            size = 10,
            color = [rp, rp],
            colorscale = 'Jet',
            cmin = 0,
            cmax = len(routes)-1,
        ), line = dict(
            width = 5,
            color = 'rgb' + str(colors.colorConverter.to_rgb(jet(norm(rp)))),
        ), hoverinfo = "text",
        text = ['<b>Bus</b>: R{}-{}<br><b>Departs</b>: {}'.format(rp,kp,round(v[d[rp][stops[rp].index(sp)][kp]])), '']

))

lo = alter_layout({
    'title'  : "Groups",
    'hovermode': "closest",
    'height' : 100+50*len(transfers),
    'yaxis'  : {'zeroline': False, 'showgrid': False, 'tickvals': range(len(groups)), 'ticktext': ['G{}'.format(g) for g in range(len(groups))], 'showticklabels': True, 'autorange': 'reversed'},
    'xaxis'  : {'autorange': False, 'range': [-1, plan_horizon+1], 'showgrid': True, 'autotick': False, 'dtick': 5, 'showticklabels': True},
})

fig = Figure(data=traces, layout=lo)
iplot(fig, filename='ISTTT23_groups')